# RNN Module

rnn 버리고 lstm으로 먼저 구현해봅니다 <br />
input : FormattedReview List <br />
output : numpy array <br />

FormattedReview
- context              : Numpy array
- context_bayes        : Tuple List
- comp_similarity      : float
- rate                 : float
- reiteration_context  : float
- reiteration_repeat   : float
- post_time            : float
- post_vip             : float
- id                   : float

- index                : int
- label                : bool

sample code from <br />
https://discuss.pytorch.org/t/example-of-many-to-one-lstm/1728/2

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable    

In [ ]:
model = nn.LSTM(in_size, classes_no, 2)
input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
output_seq, _ = model(input_seq)
last_output = output_seq[-1]

loss = nn.CrossEntropyLoss()
target = Variable(torch.LongTensor(batch_size).random_(0, classes_no-1))
err = loss(last_output, target)
err.backward()

In [18]:
input = Variable(torch.randn(3, 1, 5))
h0 = Variable(torch.randn(1, 1, 10))
c0 = Variable(torch.randn(1, 1, 10))

In [38]:
rnn = nn.LSTM(5, 10, 1)
output, hn = rnn(input, (h0, c0))
print(input)
print(output)

Variable containing:
(0 ,.,.) = 
 -0.1007  0.0854 -0.2965  0.1249 -1.0662

(1 ,.,.) = 
  0.6835  1.2083 -0.4033 -0.8664  0.4078

(2 ,.,.) = 
 -0.0301 -0.4181  0.0801  0.8633  1.7904
[torch.FloatTensor of size 3x1x5]

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.3721 -0.2455  0.4047  0.0188 -0.0347 -0.1264  0.0526 -0.1881 -0.1872

Columns 9 to 9 
  -0.1837

(1 ,.,.) = 

Columns 0 to 8 
   0.2394 -0.0665  0.3482  0.1626 -0.0491 -0.1298  0.0904 -0.0084 -0.2787

Columns 9 to 9 
  -0.2293

(2 ,.,.) = 

Columns 0 to 8 
   0.2003 -0.0439  0.2087 -0.0600  0.1815  0.0176  0.0824  0.0505 -0.0520

Columns 9 to 9 
  -0.0149
[torch.FloatTensor of size 3x1x10]



In [2]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
    
sys.meta_path.append(NotebookFinder())

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable    

input_size = 100  # word2vec k size
hidden_size = 10  # hyperparameter
num_layers = 3    # hyperparameter
p_dropout = 0.05  # hyperparameter

In [4]:
class RNN_model(nn.Module):
    def __init__(self):
        super(RNN_model, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout = p_dropout)
        
    def forward(self, formattedData, hidden=None):
        context = formattedData.context
        seq_len = context.shape[0]
        context_ = context.reshape(seq_len, 1, input_size)
        
        input = Variable(torch.from_numpy(context_), requires_grad = False)
        output, hidden = self.rnn(input, hidden)
        return output[-1]

In [5]:
import pickle

def save_object(obj, filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [1]:
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))

from Preprocessor import bookingreview
from Preprocessor import preprocessreview
from Preprocessor import wordvectormaker

NameError: name 'os' is not defined

In [12]:
bookingreview.BookingReview()

formatted_list = load_object("../Preprocessor/save_formatted_review.pkl")
test_rnn = RNN_model()
test_rnn(formatted_list[0])

TypeError: __init__() missing 7 required positional arguments: 'company', 'id', 'rate', 'context', 'post_time', 'label', and 'review_id'